In [161]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import re
from datetime import date
from datetime import datetime

# Class Detik Scraping

class dibawah merupakan proses pengambilan data dari website berita Detik.com yang dimana pemanggilan semua fungsi dibawah tersebut nantinya akan dipanggil pada fungsi [scrapingDataDetik]

In [187]:
class Detik:  
    #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    today = datetime.now()
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian):
        self.url = urlBerita
        self.keyword = pencarian
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")  
    
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult

    #fungsi buat ngambil jumlah page pada pencarian berita tersebut
    def getPage(self):
        if(self.keyword == ""):
            try:
                dateNow = today.strftime("%m/%d/%Y")
                urlGet = ("https://news.detik.com/indeks/1?date="+dateNow)
                soup = BeautifulSoup(get(urlGet).text, 'lxml')
                getAllPage= soup.find('div','pagination text-center mgt-16 mgb-16')
                page_link = getAllPage.select('a')
                for page in page_link :
                    if(page.text != "Next"):
                        lastPage = page.text
            except Exception as e:
                print(e)
        else:
            try:
                dateNow = today.strftime("%m/%d/%Y")
                urlGet = (self.url+"/search/searchall?query="+self.keyword+"&fromdatex="+dateNow+"&todatex="+dateNow)
                soup = BeautifulSoup(get(urlGet).text, 'lxml')
                getAllPage= soup.find('div','paging text_center')
                page_link = getAllPage.select('a')
                for page in page_link:
                    if(page.text != ""):
                        lastPage = page.text
            except Exception as e:
                print(e)
        return lastPage
    
    #ngambil semua url-detail pada scraping data
    def getDetailUrl(self, page):
        self.urlDetailData = []
        for page in range(page): 
            if(self.keyword == ""):
                dateNow = today.strftime("%m/%d/%Y")
                urlGet = ("https://news.detik.com/indeks/"+format(page+1)+"?date="+dateNow)
            else:
                dateNow = today.strftime("%d/%m/%Y")
                urlGet = (self.url+"/search/searchall?query="+self.keyword+"&fromdatex="+dateNow+"&todatex="+dateNow+"&page={}".format(page+1))
            indeksPage = get(urlGet)
            soup = BeautifulSoup(indeksPage.text, 'html.parser')
            contents = soup.find_all('article')
            for content in contents:
                try:
                    news_url = content.find('a', href=True).get('href')
                    self.urlDetailData.append(news_url)
                except Exception as e:
                    continue
        return self.urlDetailData
    
    #Fungsi utama buat scraping semua data tersebut
    def scrapingDataDetik(self):
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    header = soup.find('article','detail')
                    #Start cleansing judul berita solanya spacenya lebar kali
                    title = header.find('h1','detail__title').text
                    title = re.sub(r'\s+', ' ',title)
                    title = title.lstrip()
                    #End cleansing judul berita
                    author_class = header.find('div','detail__author').text.split(' ')
                    authorOne = author_class[0]
                    media = "Detik.com"
                    label = soup.find('div','page__breadcrumb')
                    label = label.find('a',href=True).text
                    author = (authorOne+" & ")+"".join(author_class[:2])
                    release_date = header.find('div', class_='detail__date').text
                    detail_text_class = soup.find('div', class_='detail__body-text')
                    texts = detail_text_class.find_all('p')
                    news_content = ' '.join([text.text for text in texts])
                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                    print(title)
                except Exception as e:
                    continue    
            else:
                continue

In [189]:
url = 'https://www.detik.com'
keyword = ''

detik = Detik(url,keyword)
detik.getDetailUrl(1)

['https://news.detik.com/berita/d-5859278/muncul-fenomena-kasus-viral-lalu-baru-diusut-kapolri-minta-ada-evaluasi',
 'https://news.detik.com/berita-jawa-timur/d-5859277/tiba-di-ngawi-jokowi-tunaikan-salat-jumat-berjemaah',
 'https://news.detik.com/berita/d-5859276/mahfud-ungkap-22-jaksa-senior-tangani-pelanggaran-ham-berat-paniai-papua',
 'https://news.detik.com/berita/d-5859275/ngajar-pakai-visa-kunjungan-wn-sudan-dideportasi-dari-sumsel',
 'https://news.detik.com/berita/d-5859274/sederet-kecelakaan-tabrak-tiang-monorel-motor-kopaja-hingga-mercy',
 'https://news.detik.com/berita-jawa-barat/d-5859272/wisatawan-keluhkan-tukang-tato-temporer-ngepruk-harga-di-bandung',
 'https://news.detik.com/berita-jawa-barat/d-5859271/pemkab-majalengka-siapkan-langkah-antisipasi-masuknya-omicron',
 'https://news.detik.com/berita/d-5859261/wn-china-probable-omicron-di-manado-tka-transit-kini-dikarantina',
 'https://news.detik.com/berita/d-5859260/omicron-masuk-ri-satgas-mari-bersama-pertahankan-kondisi-